In [15]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0.1,
)

memory = ConversationBufferMemory(llm=llm, max_token_limit=120, return_messages=True)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./files/document.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 문서를 정확하게 분석하는 전문가 입니다. 답변은 문서에 기반하여 해주세요. 모르는 내용은 모른다고 해주세요. 한국어로 답변해주세요. {context}",
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]


chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough(),
        "history": RunnableLambda(load_memory),
    }
    | prompt
    | llm
)

def invoke_chain(question):
    result = chain.invoke(question)
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)
    
question_list = ["Aaronson 은 유죄인가요?", "그가 테이블에 어떤 메시지를 썼나요?", "Julia 는 누구인가요?"]
for question in question_list:
    invoke_chain(question)

content='문서에는 Aaronson에 대한 구체적인 언급이나 그의 유죄 여부에 대한 정보가 포함되어 있지 않습니다. 따라서 Aaronson이 유죄인지에 대한 답변을 제공할 수 없습니다. 추가적인 정보가 필요합니다.' response_metadata={'token_usage': <OpenAIObject at 0x1e4a2070350> JSON: {
  "prompt_tokens": 2047,
  "completion_tokens": 51,
  "total_tokens": 2098
}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_f33667828e', 'finish_reason': 'stop', 'logprobs': None} id='run-90029e35-5b63-45aa-9f21-0333cc99c5dc-0'
content='문서에서 Winston은 테이블에 다음과 같은 메시지를 썼습니다:\n\n1. "FREEDOM IS SLAVERY"\n2. "TWO AND TWO MAKE FIVE"\n3. "GOD IS POWER"\n\n이 메시지들은 그가 자신의 생각을 기록하려고 할 때 쓴 내용입니다.' response_metadata={'token_usage': <OpenAIObject at 0x1e4a2070350> JSON: {
  "prompt_tokens": 2361,
  "completion_tokens": 66,
  "total_tokens": 2427
}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_5bd87c427a', 'finish_reason': 'stop', 'logprobs': None} id='run-513cb44b-edf4-4dc4-a325-d7316d731a47-0'
content='문서에서 Julia는 Winston의 사랑하는 사람으로 언급됩니다. 그들은 함께 자유롭게 있을 때 서로에 대한 강한 감정을 가지고 있었지만, Wins